## <center>**Molecular simulations using ASE with CP2K as Calculator**</center>"

### **AIM: Study Phosphoric acid interaction with water**


<div style="text-align:center;">
  <img src="images/problem1.png" width="800" alt="Centered image">
</div>

---
<div style="text-align:center;">
  <img src="images/problem2.png" width="800" alt="Centered image">
</div>

---

### Import required libraries

In [ ]:
import os
import numpy as np
import nglview as nv
import matplotlib.pyplot as plt
from ase import Atoms
from ase.build import molecule, add_adsorbate
from ase.io import write, read
from ase.visualize import view
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.nvtberendsen import NVTBerendsen
from ase import units
from ase.optimize import QuasiNewton, BFGS
from ase.io.trajectory import Trajectory
from ase.calculators.emt import EMT
from ase.calculators.cp2k import CP2K
from ase.build import fcc110
from ase.constraints import FixAtoms
from ase.optimize import QuasiNewton

### 1. Modelling: Solvate Phosphoric acid (H3PO4) with water

#### Load H3PO4 Molecule 

In [ ]:
# Load H3PO4 molecule
H3PO4 = read("h3po4.xyz")
nv.show_ase(H3PO4)

---

#### Solvate H3PO4

<div style="text-align:center;">
  <img src="images/packmol.png" width="700" alt="Centered image">
</div>

In [ ]:
# DOWNLOAD AND INSTALL PACKMOL Software
work_path = os.getcwd()
print(work_path)

# Change to /tmp dir and download packmol
os.chdir("/tmp/")
! wget https://github.com/m3g/packmol/archive/refs/tags/v21.0.1.tar.gz
! tar -xf v21.0.1.tar.gz

# install packmol
os.chdir("/tmp/packmol-21.0.1")
! ./configure
! make && make install > /dev/null 2>&1
os.chdir(work_path)

In [ ]:
# create a single water molecule
water = molecule('H2O')
write('water.xyz', water)

# Write packmol input file
with open('packmol.inp', 'w') as f:
    f.write(f"""
tolerance 1.5
filetype xyz
output solvated.xyz
pbc 10 10 10 

structure h3po4.xyz
  number 1
  center
  fixed 5. 5. 5. 0. 0. 0. # fix h3po4 at center
end structure

structure water.xyz
  number 32 # based on 1gm/cm3
  inside box 0. 0. 0. 10. 10. 10. 
end structure
""")

# Run packmol
! /tmp/packmol-21.0.1/packmol < packmol.inp

#### Read Solvated structure and add set cell dimensions

In [ ]:
# Read solvated structure
solvated = read('solvated.xyz')
solvated.set_pbc([True, True, True])
nv.show_ase(solvated)
#view(solvated, viewer="ngl")

In [ ]:
# setup cell dimensions # while solvating with packmol we used 10 10 10 Ang as cell dimensions.
solvated.set_cell([11, 11, 11, 90, 90, 90])
solvated.set_pbc(True)
view(solvated, viewer="ngl")

### 2. Add CP2K Calculator

In [ ]:
inp = '''
&FORCE_EVAL
  &DFT
    CHARGE 0
    # BASIS_SET_FILE_NAME BASIS_MOLOPT
    # POTENTIAL_FILE_NAME POTENTIAL
    # https://manual.cp2k.org/trunk/methods/dft/cutoff.html
    &MGRID
      NGRIDS 5
      REL_CUTOFF 50
      # CUTOFF 500
    &END MGRID
    # https://manual.cp2k.org/trunk/CP2K_INPUT/FORCE_EVAL/DFT/QS.html
    &QS
      METHOD GPW
      EPS_DEFAULT 1.0E-12
      EXTRAPOLATION ASPC
    &END QS
    # https://manual.cp2k.org/trunk/CP2K_INPUT/FORCE_EVAL/DFT/SCF.html
    &SCF
      SCF_GUESS RESTART
      EPS_SCF 1.0E-4
      # MAX_SCF 50
      CHOLESKY INVERSE
      IGNORE_CONVERGENCE_FAILURE
      &OT
        MINIMIZER DIIS
        PRECONDITIONER FULL_ALL
      &END OT
    &END SCF
    &XC
      &XC_FUNCTIONAL PBE
      &END XC_FUNCTIONAL
      &VDW_POTENTIAL
            POTENTIAL_TYPE PAIR_POTENTIAL
            &PAIR_POTENTIAL
          PARAMETER_FILE_NAME dftd3.dat
          TYPE DFTD3
              CALCULATE_C9_TERM .TRUE.
              REFERENCE_C9_TERM .TRUE.
          REFERENCE_FUNCTIONAL PBE
            &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
    &END XC
  &END DFT
&END FORCE_EVAL
'''

#### Assign calculator to atoms

In [ ]:
# === Step 4: Set up CP2K calculator ===
calc = CP2K(label='Solvate-H3PO4',
            cutoff=250,
            basis_set='SZV-MOLOPT-GTH',
            pseudo_potential='auto',
            charge=0,
            command="/custom_software_rocky/additional/cp2k/exe/local/cp2k_shell.psmp",
            xc='PBE',
            print_level='LOW',
            max_scf=20,
            inp=inp)


solvated.calc = calc
# atoms.set_calculator(EMT())

###  <center>PLEASE DO NOT RUN BELOW CELLS </center>
###  <center>DFT simulations are computationally expensive, so we will submit a slurm job</center>

### 3. Run Geometry Optimization 

In [ ]:
# BFGS optimizer
opt = BFGS(solvated, trajectory="optimized.traj", append_trajectory=False, logfile="opt.log")
energies = []

def print_status(a=solvated):
    epot = a.get_potential_energy()
    ekin = a.get_kinetic_energy()
    energies.append(epot+ekin)
    print(f' Energy | Epot = {epot:.3f} eV | Ekin = {ekin:.3f} eV | Etot = {epot+ekin:.3f} eV')

# One can attach functions to modify the output
opt.attach(print_status, interval=1)
print("Running optimization...")
opt.run(fmax=0.02, steps=2)
print("Simulation complete.")

#### Energy profile: geometry optimization

In [ ]:
plt.figure()
plt.plot(range(len(energies)), energies, marker='o')
plt.xlabel('Optimization Step')
plt.ylabel('Energy (eV)')
plt.title('Geometry Optimization of Solvated benzene')
plt.savefig("optimization_energy.png")
plt.show()

#### Read geometry optimized trajectory 

In [ ]:
frames = read('optimized.traj', index=':')
optimized = Trajectory('optimized.traj')[-1]
view(frames, viewer="ngl")

### 4. Run Molecular Dynamics 

#### Create MD calculator

In [ ]:
calc_md = CP2K(label='md',
            cutoff=500,
            basis_set='SZV-MOLOPT-GTH',
            pseudo_potential='auto',
            charge=0,
            xc='PBE',
            command="/custom_software_rocky/additional/cp2k/exe/local/cp2k_shell.psmp",
            print_level='LOW',
            max_scf=50,
            inp=inp)

optimized.calc = calc_md

In [ ]:
MaxwellBoltzmannDistribution(optimized, temperature_K=300)
mddyn = NVTBerendsen(optimized, timestep=0.5 * units.fs, temperature_K=300, taut=10 * units.fs, trajectory='md.traj', logfile='md.log', append_trajectory=True)
print("Running MD simulation...")
mddyn_eql.run(2)  # 200 steps = 100 fs
print("Simulation complete.")

# md_dyn = Langevin(opt.atoms, timestep=0.5 * units.fs, temperature_K=300.0, friction=0.01 / units.fs, trajectory='MDrun.traj')  # 0.5 fs
# mddyn = NVTBerendsen(opt.atoms, timestep = 0.5 * units.fs, temperature_K= 300.0, taut=0.5*1000*units.fs)
# mddyn = VelocityVerlet(optimized_snapshot, timestep = 0.5 * units.fs) 
# def print_status(a=atoms):
#    epot = a.get_potential_energy()
#    ekin = a.get_kinetic_energy()
#    print(f'Energy | Epot = {epot:.3f} eV | Ekin = {ekin:.3f} eV | Etot = {epot+ekin:.3f} eV')
# mddyn.attach(print_status, interval=1)

### 5. Postprocessing

#### Visualize trajectory

In [ ]:
frames = read('md.traj', index=':')
view(frames, viewer="ngl")

#### Energy vs MD Step

In [ ]:
plt.figure()
plt.plot(range(len(energies)), energies, marker='o')
plt.xlabel('MD Step')
plt.ylabel('Energy (eV)')
plt.title('MD simulation of Solvated H3PO4')
plt.savefig("MD_energyprofile.png")
plt.show()